# Foundations of Artificial Intelligence and Machine Learning
## A Program by IIIT-H and TalentSprint

#### To be done in the Lab

The objective of this experiment is to perform sentimental analysis.

In this experiment we will be using twitter dataset as training data and crawled realtime tweets for testing. 

The Ground truth is 1 for positive tweet and 0 for negative tweet.

Few examples of positive and negative tweets are:

**Few Positive Tweets: **
1.  @Msdebramaye I heard about that contest! Congrats girl!!
2. UNC!!! NCAA Champs!! Franklin St.: I WAS THERE!! WILD AND CRAZY!!!!!! Nothing like it...EVER http://tinyurl.com/49955t3

**Few Negative Tweets:**
1. no more taking Irish car bombs with strange Australian women who can drink like rockstars...my head hurts.
2. Just had some bloodwork done. My arm hurts

### Data Source

https://www.kaggle.com/c/twitter-sentiment-analysis2/data


## Exercise 1: (2 marks)

The first exercise is cleaning the tweets.
Perform preprocessing as required.

Complete the functon : preprocess_tweets 

Input or Arguement to the function : tweet as a string 

Return value: processed tweet as string 

Hint: Use regular expressions
* convert the all the cases into lower case
  + look at lower()
* Replace any urls with the word "URL"
  + Hint : 
      - re.sub('((www\.[^\s]+)|(https?://[^\s]+))','URL',"Tweet") (re is python regular expression package)
* convert the username to "AT_USER", consider any word that starts with @ as user name
  + Hint : 
      - re.sub('@[^\s]+','AT_USER',"Tweet")
* Remove multiple whitespaces with a single white space
  + Hint :
      - re.sub('[\s]+', ' ', tweet)
* Replace hashtag words (#word) with just the words (word)
  + Hint : 
      - re.sub(r'#([^\s]+)', r'\1', "tweet")
      
* TEST CASE :
    + given the tweet "@V_DEL_ROSSI: Me         #dragging myself to the gym https://t.co/cOjM0mBVeY"
    + output should be "AT_USER me dragging myself to the gym URL"

In [1]:
import re
import pandas as pd
import numpy as np
from sklearn import feature_extraction, svm
from sklearn.metrics import confusion_matrix, precision_recall_fscore_support
from sklearn.model_selection import train_test_split

In [2]:
def preprocess_tweets(tweet):
    #Code here
    #print("Original: " ,tweet)
    #Convert to lower
    tweet = tweet.lower()
    #print("To lower: ", tweet)
    
    #Replace URL
    tweet = re.sub('((www.[^\s]+)|(https?://[^\s]+))','URL',tweet)
    #print("Removing URL : ", tweet)
    
    #Replace @user
    tweet = re.sub('@[^\s]+','AT_USER',tweet)
    #print("Removing @user : ", tweet)
    
    #Replace multiple whitespace
    tweet = re.sub('[\s]+', ' ', tweet)
    #print("Removing multiple space : ", tweet)
    
    #Replace hashtag
    tweet = re.sub(r'#([^\s]+)', r'\1', tweet)
    #print("Removing # : ", tweet)
    
    return tweet

# Test it out
tweet = preprocess_tweets("@V_DEL_ROSSI: Me #dragging myself to     the gym   https://t.co/cOjM0mBVeY")

## Exercise 2: (3 marks)

Tokenize the processed tweets to make a tweet into a list of words and make sure that no punctuations are returned. so that it can be used in the next steps to represent the tweet as a feature vector. Remove the Stops words, if necessary

Complete the functon : word_tokenizer 

Input or Arguement to the function : processed tweet

Return value: list of words without any punctuations

TEST CASE :

Given an input :
    "Neither Man, nor machine can replace its creator. really?."
    
Result : 
    ['neither', 'man', 'nor', 'machine', 'replace', 'creator', 'hahaha']

In [3]:
stopWords = pd.read_csv('stopwords.txt')["a"].values.tolist()
stopWords = set(stopWords)
stopWords

posWords = pd.read_csv('positive-words.txt')["a+"].values.tolist()
posWords = set(posWords)
posWords

negWords = pd.read_csv('negative-words.txt')["S2-faced"].values.tolist()
negWords = set(negWords)
negWords

{'bereave',
 'confusion',
 'forsake',
 'overreach',
 'fatcats',
 'abolish',
 'agonies',
 'disconsolation',
 'subtract',
 'tumbled',
 'taut',
 'bellicose',
 'impedance',
 'fury',
 'scar',
 'scoundrel',
 'unbelievably',
 'uncooperative',
 'apathy',
 'aimless',
 'underpaid',
 'sarcastically',
 'sly',
 'bungling',
 'unnatural',
 'unwillingly',
 'annihilate',
 'cheating',
 'imaginary',
 'brashly',
 'hobble',
 'far-fetched',
 'tramp',
 'inconsequently',
 'upsetting',
 'showdown',
 'damnable',
 'carelessness',
 'suppress',
 'disaffect',
 'damned',
 'notoriety',
 'mists',
 'embarrassingly',
 'deplete',
 'inconsistent',
 'excruciating',
 'depression',
 'malicious',
 'horrendous',
 'rotten',
 'evasion',
 'fictitious',
 'issue',
 'deceptive',
 'eschew',
 'insultingly',
 'discomfititure',
 'instable',
 'muddy',
 'destroy',
 'unviewable',
 'angriness',
 'mess',
 'concessions',
 'congestion',
 'flaky',
 'sting',
 'deride',
 'disparagingly',
 'knock',
 'reticent',
 'toil',
 'botch',
 'scratch',
 'unt

In [4]:
def word_tokenizer(tweet):
    
    tweet_words = re.split(r"[\W\s_]+", tweet)
    tweet_words = list(filter(None, tweet_words))
    
    tokenized_tweet = [word for word in tweet_words if word not in stopWords]
    return tokenized_tweet

tweet = "Neither Man, nor machine can replace its creator. really?. hahaha"
tweet = preprocess_tweets(tweet)
word_tokenizer(tweet)

['neither', 'nor', 'machine', 'replace', 'creator', 'hahaha']

## Exercise 3: (5 marks)

Using the list of words from the above the step, 
* represent the tweet as a feature vector using bag of words

Hint : counts of postive/negative/neutral words as three features can also be used

In [5]:
def getfeaturevector(tokenized_tweet):
#Code here
    pos, neg, neu = 0,0,0
    for x in tokenized_tweet:
        if x in posWords:
            pos += 1
        elif x in negWords:
            neg += 1
        else:
            neu += 1
    
        
    return [pos, neg, neu]

## Exercise 4: (Marks : 5 ) 


Load the given training data and use the above functions you created to process, to tokenise and to get feature vector.

Considering the feature vector as input to the classifier, Train a classifier to classify the sentiment of the tweet correctly.

Divide the training data into two sets, to validate your classifier

In [6]:
#Code here


train = pd.read_csv("twitter_train.csv", encoding = "ISO-8859-1", index_col=["ItemID"])
train.head()
train["Feature"] = train["SentimentText"].apply(lambda x: getfeaturevector(word_tokenizer(preprocess_tweets(x))))
train.head()

,Sentiment,SentimentText,Feature
ItemID,,,
1,0,is so sad for my APL frie...,"[0, 1, 2]"
2,0,I missed the New Moon trail...,"[0, 1, 2]"
3,1,omg its already 7:30 :O,"[0, 0, 3]"
4,0,.. Omgaga. Im sooo im gunna CRy. I'...,"[0, 1, 13]"
5,0,i think mi bf is cheating on me!!! ...,"[0, 1, 2]"


In [7]:
train.drop(["SentimentText"], inplace=True, axis=1)
train ["pos"] = train.Feature.apply(lambda x: x[0])
train ["neg"] = train.Feature.apply(lambda x: x[1])
train ["neu"] = train.Feature.apply(lambda x: x[2])
train.drop(["Feature"], inplace=True, axis=1)
train.head()

,Sentiment,pos,neg,neu
ItemID,,,,
1,0,0,1,2
2,0,0,1,2
3,1,0,0,3
4,0,0,1,13
5,0,0,1,2


In [8]:
X, Y = train.iloc[:10000, 1:], train.iloc[:10000,0]
train_x, valid_x, train_y, valid_y = train_test_split(X, Y, test_size=0.2)

In [12]:
classifier = svm.SVC()
classifier.fit(train_x, train_y)
print(classifier.score(train_x, train_y))
print(classifier.score(valid_x, valid_y))

0.68025
0.6955


## Exercise 5: (Marks : 5)

#### Twitter crawling using tweepy

Use tweepy to get the tweets on real time, which is used as test data for the classifier.

## Requirements: 

Twitter account

Create a twitter account if you don't have one by going to the link given below:

https://twitter.com/i/flow/signup

## Installation

Tweepy: tweepy is the python client for the official Twitter API.
Install it using following pip command:

In [ ]:
!pip install tweepy

The tweets need to be gathered so as to perform Sentiment analysis on those tweets. They can be fetched from Twitter using the Twitter API. 

In order to fetch tweets through Twitter API, one needs to register an App through their twitter account. Follow these steps for the same:
<ul>
<li>Open the link given below to create a App through the twitter account.
    https://apps.twitter.com
<li>click the button: ‘Create New App’
<li>Fill the application details. You can leave the callback url field empty.
<li>Once the app is created, you will be redirected to the app page.
<li>Open the ‘Keys and Access Tokens’ tab.
<li>Copy ‘Consumer Key’, ‘Consumer Secret’, ‘Access token’ and ‘Access Token Secret’.
</ul>

In [ ]:
#Replace with your ‘Consumer Key’, ‘Consumer Secret’, ‘Access token’ and ‘Access Token Secret’ below. 

consumer_key = 'XXXXXXXXX'
consumer_secret = 'XXXXXXXXXX'
access_token = 'XXXXXXXXXXXXXX'
access_secret = 'XXXXXXXXXXXXX'

Run the code below to authenticate your application.

In [ ]:
import tweepy

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)

api = tweepy.API(auth)    

## Tweepy Cursor

The below code gives the search results from twitter for the search string passed to the keyword arguement "q" in the tweepy.Cursor. The number passed to the items method of tweepy.Cursor indicates that it gives 100 such tweets, if available

In [ ]:
for i in tweepy.Cursor(api.search, q='searchme', lang = 'en', full_text=True).items(100):
    print(i._json['text'])
    #print(processTweet(i._json['text']), end='\n\n\n')

Also apply the preprocessing steps and obtain the feature vectors for the crawled twitter data.
Classify the crawled tweets by passing its feature vector to the trained classifier.

In [ ]:
##Your code here